In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import subprocess
import pdb
import time
import datetime
import math
import copy
import random
import _pickle as cPickle
from collections import defaultdict
import matplotlib.pyplot as plt

%matplotlib inline
# %matplotlib nbagg

from six.moves import zip_longest
import numpy as np
import pandas as pd
from scipy.stats import hmean

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

# load data & set config

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 1000, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 3000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
# flags.DEFINE_string('opt', 'Adam', 'optimizer')
flags.DEFINE_float('lr', 0.01, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 20, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [5]:
def get_batches(instances, batch_size, iterator=False):
    iter_instances = iter(instances)
    n_batch = len(instances)//batch_size
    
    batches = [(i_batch, [next(iter_instances) for i_doc in range(batch_size)]) for i_batch in range(n_batch)]
    
    if iterator: batches = iter(batches)
    return batches

train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)

In [6]:
flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

In [7]:
def debug_shape(variables, model):
    sample_batch = dev_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

#  model 

## doubly rnn

In [8]:
class DoublyRNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        
        self.ancestral_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='ancestral')
        self.fraternal_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='fraternal')
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, name='hidden')
        
        self.output_layer=output_layer
        
    def __call__(self, state_ancestral, state_fraternal, reuse=True):
        with tf.variable_scope('input', reuse=reuse):
            state_ancestral = self.ancestral_layer(state_ancestral)
            state_fraternal = self.fraternal_layer(state_fraternal)

        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state_ancestral + state_fraternal)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [9]:
def doubly_rnn(dim_hidden, tree_idxs, initial_state_parent=None, initial_state_sibling=None, output_layer=None, name=''):
    outputs, states_parent = {}, {}
    
    with tf.variable_scope(name, reuse=False):
        doubly_rnn_cell = DoublyRNNCell(dim_hidden, output_layer)

        if initial_state_parent is None: 
            initial_state_parent = doubly_rnn_cell.get_initial_state('init_state_parent')
#             initial_state_parent = doubly_rnn_cell.get_zero_state('init_state_parent')
        if initial_state_sibling is None: 
#             initial_state_sibling = doubly_rnn_cell.get_initial_state('init_state_sibling')
            initial_state_sibling = doubly_rnn_cell.get_zero_state('init_state_sibling')
        output, state_sibling = doubly_rnn_cell(initial_state_parent, initial_state_sibling, reuse=False)
        outputs[0], states_parent[0] = output, state_sibling

        for parent_idx, child_idxs in tree_idxs.items():
            state_parent = states_parent[parent_idx]
            state_sibling = initial_state_sibling
            for child_idx in child_idxs:
                output, state_sibling = doubly_rnn_cell(state_parent, state_sibling)
                outputs[child_idx], states_parent[child_idx] = output, state_sibling

    return outputs, states_parent

## RNN

In [10]:
class RNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='hidden')
        self.output_layer=output_layer
        
    def __call__(self, state, reuse=True):
        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [11]:
def rnn(dim_hidden, max_depth, initial_state=None, output_layer=None, name=''):
    outputs, states_hidden = [], []
    
    with tf.variable_scope(name, reuse=False):
        rnn_cell = RNNCell(dim_hidden, output_layer)

        if initial_state is not None: 
            state_hidden = initial_state
        else:
            state_hidden = rnn_cell.get_initial_state('init_state')
#             state_hidden = rnn_cell.get_zero_state('init_state_parent')
        
        for depth in range(max_depth):
            if depth == 0:                
                output, state_hidden = rnn_cell(state_hidden, reuse=False)
            else:
                output, state_hidden = rnn_cell(state_hidden, reuse=True)
            outputs.append(output)
            states_hidden.append(state_hidden)

    outputs = tf.concat(outputs, 1)
    states_hidden = tf.concat(states_hidden, 0)
    return outputs, states_hidden

## nCRP model

In [56]:
class Model():
    def __init__(self, config, tree_idxs):
        def get_depth(parent_idx=0, tree_depth=None, depth=1):
            if tree_depth is None: tree_depth={0: depth}

            child_idxs = tree_idxs[parent_idx]
            depth +=1
            for child_idx in child_idxs:
                tree_depth[child_idx] = depth
                if child_idx in tree_idxs: get_depth(child_idx, tree_depth, depth)
            return tree_depth
        
        self.config = config
        
        self.t_variables = {}
        
        self.tree_idxs = tree_idxs
        self.topic_idxs = [0] + [idx for child_idxs in tree_idxs.values() for idx in child_idxs]
        self.child_to_parent_idxs = {child_idx: parent_idx for parent_idx, child_idxs in self.tree_idxs.items() for child_idx in child_idxs}
        self.tree_depth = get_depth()
        self.n_depth = max(self.tree_depth.values())
        
        self.build()
        
    def build(self):
        def nCRP(tree_sticks_topic):
            tree_prob_topic = {}
            tree_prob_leaf = {}
            # calculate topic probability and save
            tree_prob_topic[0] = 1.

            for parent_idx, child_idxs in self.tree_idxs.items():
                rest_prob_topic = tree_prob_topic[parent_idx]
                for child_idx in child_idxs:
                    stick_topic = tree_sticks_topic[child_idx]
                    if child_idx == child_idxs[-1]:
                        prob_topic = rest_prob_topic * 1.
                    else:
                        prob_topic = rest_prob_topic * stick_topic

                    if not child_idx in self.tree_idxs: # leaf childs
                        tree_prob_leaf[child_idx] = prob_topic
                    else:
                        tree_prob_topic[child_idx] = prob_topic

                    rest_prob_topic -= prob_topic
            return tree_prob_leaf

        def sbp(sticks_depth, max_depth):
            prob_depth_list = []
            rest_prob_depth = 1.
            for depth in range(max_depth):
                stick_depth = tf.expand_dims(sticks_depth[:, depth], 1)
                if depth == max_depth -1:
                    prob_depth = rest_prob_depth * 1.
                else:
                    prob_depth = rest_prob_depth * stick_depth
                prob_depth_list.append(prob_depth)
                rest_prob_depth -= prob_depth

            prob_depth = tf.concat(prob_depth_list, 1)
            return prob_depth
       
        def get_prob_topic(tree_prob_leaf, prob_depth):
            def get_ancestor_idxs(leaf_idx, ancestor_idxs = None):
                if ancestor_idxs is None: ancestor_idxs = [leaf_idx]
                parent_idx = self.child_to_parent_idxs[leaf_idx]
                ancestor_idxs += [parent_idx]
                if parent_idx in self.child_to_parent_idxs: get_ancestor_idxs(parent_idx, ancestor_idxs)
                return ancestor_idxs[::-1]
            
            tree_prob_topic = defaultdict(float)
            leaf_ancestor_idxs = {leaf_idx: get_ancestor_idxs(leaf_idx) for leaf_idx in tree_prob_leaf}
            for leaf_idx, ancestor_idxs in leaf_ancestor_idxs.items():
                prob_leaf = tree_prob_leaf[leaf_idx]
                for i, ancestor_idx in enumerate(ancestor_idxs):
                    prob_ancestor = prob_leaf * tf.expand_dims(prob_depth[:, i], -1)
                    tree_prob_topic[ancestor_idx] += prob_ancestor
            prob_topic = tf.concat([tree_prob_topic[topic_idx] for topic_idx in self.topic_idxs], -1)
            return prob_topic
        
        def get_tree_topic_bow(tree_topic_embeddings):
            def softmax_with_temperature(logits, axis=None, name=None, temperature=1.):
                if axis is None:
                    axis = -1
                return tf.exp(logits / temperature) / tf.reduce_sum(tf.exp(logits / temperature), axis=axis)

            tree_topic_bow = {}
            for topic_idx, depth in self.tree_depth.items():
                topic_embedding = tree_topic_embeddings[topic_idx]
                temperature = tf.constant(1. ** (1./depth), dtype=tf.float32)
                logits = tf.matmul(topic_embedding, self.bow_embeddings, transpose_b=True)
                tree_topic_bow[topic_idx] = softmax_with_temperature(logits, axis=-1, temperature=temperature)
            return tree_topic_bow
        
        def get_topic_loss_reg(tree_topic_embeddings):
            def get_tree_mask_reg(all_child_idxs):        
                tree_mask_reg = np.zeros([len(all_child_idxs), len(all_child_idxs)], dtype=np.float32)
                for parent_idx, child_idxs in self.tree_idxs.items():
#                     neighbor_idxs = child_idxs + [parent_idx] if parent_idx in all_child_idxs else child_idxs
                    neighbor_idxs = child_idxs
                    for neighbor_idx1 in neighbor_idxs:
                        for neighbor_idx2 in neighbor_idxs:
                            neighbor_index1 = all_child_idxs.index(neighbor_idx1)
                            neighbor_index2 = all_child_idxs.index(neighbor_idx2)
                            tree_mask_reg[neighbor_index1, neighbor_index2] = tree_mask_reg[neighbor_index2, neighbor_index1] = 1.
                return tree_mask_reg
            
            all_child_idxs = list(self.child_to_parent_idxs.keys())
            self.diff_topic_embeddings = tf.concat([tree_topic_embeddings[child_idx] - tree_topic_embeddings[self.child_to_parent_idxs[child_idx]] for child_idx in all_child_idxs], axis=0)
            diff_topic_embeddings_norm = self.diff_topic_embeddings / tf.norm(self.diff_topic_embeddings, axis=1, keepdims=True)
            self.topic_dots = tf.clip_by_value(tf.matmul(diff_topic_embeddings_norm, tf.transpose(diff_topic_embeddings_norm)), -1., 1.)        

            self.tree_mask_reg = get_tree_mask_reg(all_child_idxs)
            self.topic_losses_reg = tf.square(self.topic_dots - tf.eye(len(all_child_idxs))) * self.tree_mask_reg
            self.topic_loss_reg = tf.reduce_sum(self.topic_losses_reg) / tf.reduce_sum(self.tree_mask_reg)
            return self.topic_loss_reg
        
#         def get_topic_loss_reg(tree_topic_embeddings):
#             def get_tree_mask_reg(all_child_idxs):        
#                 tree_mask_reg = np.zeros([len(all_child_idxs), len(all_child_idxs)], dtype=np.float32)
#                 for parent_idx, child_idxs in self.tree_idxs.items():
#                     neighbor_idxs = child_idxs + [parent_idx] if parent_idx in all_child_idxs else child_idxs
#                     for neighbor_idx1 in neighbor_idxs:
#                         for neighbor_idx2 in neighbor_idxs:
#                             neighbor_index1 = all_child_idxs.index(neighbor_idx1)
#                             neighbor_index2 = all_child_idxs.index(neighbor_idx2)
#                             tree_mask_reg[neighbor_index1, neighbor_index2] = tree_mask_reg[neighbor_index2, neighbor_index1] = 1.
#                 return tree_mask_reg
            
#             self.topic_losses_reg = []
#             for parent_idx, child_idxs in self.tree_idxs.items():
#                 diff_topic_embeddings = tf.concat([tree_topic_embeddings[child_idx] - tree_topic_embeddings[parent_idx] for child_idx in child_idxs], axis=0)
#                 diff_topic_embeddings_norm = diff_topic_embeddings / tf.norm(diff_topic_embeddings, axis=1, keepdims=True)
#                 topic_dots = tf.clip_by_value(tf.matmul(diff_topic_embeddings_norm, tf.transpose(diff_topic_embeddings_norm)), -1., 1.)        

#                 mean_angles = tf.asin(tf.sqrt(tf.clip_by_value(tf.linalg.det(topic_dots), 0, 1)))
#                 var_angles = tf.square(tf.constant(np.pi/2., dtype=tf.float32)-mean_angles)

#                 topic_loss_reg = var_angles - mean_angles
#                 self.topic_losses_reg.append(topic_loss_reg)

#             self.topic_loss_reg = tf.reduce_sum(self.topic_losses_reg)
#             return self.topic_loss_reg
           
        # -------------- Build Model --------------
        tf.reset_default_graph()
        
        self.t_variables['bow'] = tf.placeholder(tf.float32, [None, self.config.dim_bow])
        self.t_variables['keep_prob'] = tf.placeholder(tf.float32)
        
        # encode bow
        with tf.variable_scope('topic/enc', reuse=False):
            hidden_bow_ = tf.layers.Dense(units=self.config.dim_hidden_bow, activation=tf.nn.tanh, name='hidden_bow')(self.t_variables['bow'])
            hidden_bow = tf.layers.Dropout(self.t_variables['keep_prob'])(hidden_bow_)
            means_bow = tf.layers.Dense(units=self.config.dim_latent_bow, name='mean_bow')(hidden_bow)
            logvars_bow = tf.layers.Dense(units=self.config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_bow')(hidden_bow)
            latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors
            prob_layer = lambda h: tf.nn.sigmoid(tf.matmul(latents_bow, h, transpose_b=True))

            tree_sticks_topic, tree_states_sticks_topic = doubly_rnn(self.config.dim_latent_bow, self.tree_idxs, output_layer=prob_layer, name='sticks_topic')
            tree_prob_leaf = nCRP(tree_sticks_topic)
            self.tree_prob_leaf = tree_prob_leaf
#             prob_depth = tf.layers.Dense(units=self.n_depth, activation=tf.nn.softmax, name='prob_depth')(latents_bow) # inference of topic probabilities
            sticks_depth, _ = rnn(config.dim_latent_bow, self.n_depth, output_layer=prob_layer, name='prob_depth')
            prob_depth = sbp(sticks_depth, self.n_depth)
            self.prob_depth = prob_depth

            prob_topic = get_prob_topic(tree_prob_leaf, prob_depth)
            self.prob_topic = prob_topic # n_batch x n_topic

        # decode bow
        with tf.variable_scope('shared', reuse=False):
            self.bow_embeddings = tf.get_variable('emb', [self.config.dim_bow, self.config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

        with tf.variable_scope('topic/dec', reuse=False):
        #     tree_topic_embeddings, tree_states_topic_embeddings = doubly_rnn(self.config.dim_emb, self.tree_idxs, name='emb_topic')
            emb_layer = lambda h: tf.layers.Dense(units=self.config.dim_emb, name='output')(tf.nn.tanh(h))
            tree_topic_embeddings, tree_states_topic_embeddings = doubly_rnn(self.config.dim_emb, self.tree_idxs, output_layer=emb_layer, name='emb_topic')
#             topic_embeddings = tf.get_variable('topic_emb', [len(self.topic_idxs), self.config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics
#             tree_topic_embeddings = {topic_idx: tf.expand_dims(topic_embeddings[self.topic_idxs.index(topic_idx)], 0) for topic_idx in self.topic_idxs}

            self.tree_topic_bow = get_tree_topic_bow(tree_topic_embeddings) # bow vectors for each topic

            topic_bow = tf.concat([self.tree_topic_bow[topic_idx] for topic_idx in self.topic_idxs], 0) # KxV
            self.topic_bow = topic_bow
            logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution N_Batch x  V
            self.logits_bow = logits_bow
            
        # define losses
        self.topic_losses_recon = -tf.reduce_sum(tf.multiply(self.t_variables['bow'], logits_bow), 1)
        self.topic_loss_recon = tf.reduce_mean(self.topic_losses_recon) # negative log likelihood of each words

        self.topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std
        
        self.topic_embeddings = tf.concat([tree_topic_embeddings[topic_idx] for topic_idx in self.topic_idxs], 0) # temporary
        self.topic_loss_reg = get_topic_loss_reg(tree_topic_embeddings)

#         self.topic_embeddings = tf.concat([tree_topic_embeddings[topic_idx] for topic_idx in self.topic_idxs], 0)
#         topic_embeddings_norm = self.topic_embeddings / tf.norm(self.topic_embeddings, axis=1, keepdims=True)
#         self.topic_dots = tf.clip_by_value(tf.matmul(topic_embeddings_norm, tf.transpose(topic_embeddings_norm)), -1., 1.)
#         self.mean_angles = tf.asin(tf.sqrt(tf.linalg.det(self.topic_dots)))
#         self.var_angles = tf.square(tf.constant(np.pi/2., dtype=tf.float32)-self.mean_angles)
#         self.topic_loss_reg = self.var_angles - self.mean_angles

        self.global_step = tf.Variable(0, name='global_step',trainable=False)

        self.loss = self.topic_loss_recon + self.topic_loss_kl + self.config.reg * self.topic_loss_reg

        # define optimizer
        if self.config.opt == 'Adam':
            optimizer = tf.train.AdamOptimizer(self.config.lr)
        elif self.config.opt == 'Adagrad':
            optimizer = tf.train.AdagradOptimizer(self.config.lr)

        self.grad_vars = optimizer.compute_gradients(self.loss)
        self.clipped_grad_vars = [(tf.clip_by_value(grad, -self.config.grad_clip, self.config.grad_clip), var) for grad, var in self.grad_vars]
        self.opt = optimizer.apply_gradients(self.clipped_grad_vars, global_step=self.global_step)

        # monitor
        self.n_bow = tf.reduce_sum(self.t_variables['bow'], 1)
        self.topic_ppls = tf.divide(self.topic_losses_recon, tf.maximum(1e-5, self.n_bow))
        self.topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices
    
        # growth criteria
#         self.dist_bow = -tf.matmul(self.t_variables['bow'], tf.log(topic_bow), transpose_b=True)
#         self.rads_bow = tf.divide(tf.multiply(self.dist_bow, prob_topic), tf.expand_dims(self.n_bow, -1))
        self.n_topics = tf.multiply(tf.expand_dims(self.n_bow, -1), prob_topic)
        
        self.arcs_bow = tf.acos(tf.matmul(tf.linalg.l2_normalize(self.bow_embeddings, axis=-1), tf.linalg.l2_normalize(self.topic_embeddings, axis=-1), transpose_b=True)) # n_vocab x n_topic
        self.rads_bow = tf.multiply(tf.matmul(self.t_variables['bow'], self.arcs_bow), self.prob_topic) # n_batch x n_topic
    
    def get_feed_dict(self, batch, mode='train'):
        bow = np.array([instance.bow for instance in batch]).astype(np.float32)
        keep_prob = self.config.keep_prob if mode == 'train' else 1.0
        feed_dict = {
                    self.t_variables['bow']: bow, 
                    self.t_variables['keep_prob']: keep_prob
        }
        return  feed_dict

# run

In [57]:
def get_loss(sess, batches, model):
    losses = []
    ppl_list = []
    rads_bow_list = []
    prob_topic_list = []
    n_bow_list = []
    n_topics_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, rads_bow_batch, prob_topic_batch, n_bow_batch, n_topics_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, model.rads_bow, model.prob_topic, model.n_bow, model.n_topics], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        rads_bow_list.append(rads_bow_batch)
        prob_topic_list.append(prob_topic_batch)
        n_bow_list.append(n_bow_batch)
        n_topics_list.append(n_topics_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    
    probs_topic = np.concatenate(prob_topic_list, 0)
    
    n_bow = np.concatenate(n_bow_list, 0)
    n_topics = np.concatenate(n_topics_list, 0)
    probs_topic_mean = np.sum(n_topics, 0) / np.sum(n_bow)
    
    rads_bow = np.concatenate(rads_bow_list, 0)
    rads_bow_mean = np.cos(np.sum(rads_bow, 0) / np.sum(n_topics, 0))
    
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, ppl_mean, rads_bow_mean, probs_topic_mean

In [58]:
def print_topic_sample(tree_idxs, sess=None, model=None, topic_rad_bow=None, topic_prob_topic=None, parent_idx=0, topics_freq_bow_idxs=None, depth = 0):
    if topics_freq_bow_idxs is None:
        topics_freq_bow_idxs = bow_idxs[sess.run(model.topics_freq_bow_indices)]
        topic_freq_bow_idxs = topics_freq_bow_idxs[model.topic_idxs.index(parent_idx)]
        rad_bow = topic_rad_bow[parent_idx]
        prob_topic = topic_prob_topic[parent_idx]
        print(parent_idx, 'R: %.3f' % rad_bow, 'P: %.3f' % prob_topic, ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
    
    child_idxs = tree_idxs[parent_idx]
    depth += 1
    for child_idx in child_idxs:
        topic_freq_bow_idxs = topics_freq_bow_idxs[model.topic_idxs.index(child_idx)]
        rad_bow = topic_rad_bow[child_idx]
        prob_topic = topic_prob_topic[child_idx]
        print('  '*depth, child_idx, 'R: %.2f' % rad_bow, 'P: %.3f' % prob_topic, ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        
        if child_idx in tree_idxs: print_topic_sample(tree_idxs, model=model, topic_rad_bow=topic_rad_bow, topic_prob_topic=topic_prob_topic, parent_idx=child_idx, topics_freq_bow_idxs=topics_freq_bow_idxs, depth=depth)

In [59]:
def print_topic_specialization(sess, model):
    topics_bow = sess.run(model.topic_bow)
    norm_bow = np.sum([instance.bow for instance in instances_test], 0)
    topics_vec = topics_bow / np.linalg.norm(topics_bow, axis=1, keepdims=True)
    norm_vec = norm_bow / np.linalg.norm(norm_bow)

    topics_spec = 1 - topics_vec.dot(norm_vec)

    depth_topic_idxs = defaultdict(list)
    for topic_idx, depth in model.tree_depth.items():
        depth_topic_idxs[depth].append(topic_idx)

    for depth, topic_idxs in depth_topic_idxs.items():
        topic_indices = np.array([model.topic_idxs.index(topic_idx) for topic_idx in topic_idxs])
        depth_spec = np.mean(topics_spec[topic_indices])
        print(depth, depth_spec)    

In [127]:
def print_hierarchical_affinity(sess, model):
    def get_cos_sim(parent_to_child_idxs):
        parent_child_bows = {parent_idx: np.concatenate([normed_tree_topic_bow[child_idx] for child_idx in child_idxs], 0) for parent_idx, child_idxs in parent_to_child_idxs.items()}
        cos_sim = np.mean([np.mean(normed_tree_topic_bow[parent_idx].dot(child_bows.T)) for parent_idx, child_bows in parent_child_bows.items()])
        return cos_sim    
    
    tree_topic_bow = sess.run(model.tree_topic_bow)
    normed_tree_topic_bow = {topic_idx: topic_bow/np.linalg.norm(topic_bow) for topic_idx, topic_bow in tree_topic_bow.items()}

    third_child_idxs = [child_idx for child_idx, depth in model.tree_depth.items() if depth==3]
    second_parent_to_child_idxs = {parent_idx:child_idxs for parent_idx, child_idxs in model.tree_idxs.items() if model.tree_depth[parent_idx] == 2}
    second_parent_to_unchild_idxs = {parent_idx: [child_idx for child_idx in third_child_idxs if child_idx not in child_idxs] for parent_idx, child_idxs in second_parent_to_child_idxs.items()}
    
    child_cos_sim = get_cos_sim(second_parent_to_child_idxs)
    unchild_cos_sim = get_cos_sim(second_parent_to_unchild_idxs)
    print('child %.3f, not-child: %.3f' % (child_cos_sim, unchild_cos_sim))

In [60]:
def get_recur_topic_idxs(model, parent_idx, recur_topic_idxs = None):
    if recur_topic_idxs is None: recur_topic_idxs = [parent_idx]

    if parent_idx in model.tree_idxs:
        child_idxs = model.tree_idxs[parent_idx]
        recur_topic_idxs += child_idxs
        for child_idx in child_idxs:
            if child_idx in model.tree_idxs: get_recur_topic_idxs(model, child_idx, recur_topic_idxs)
    return recur_topic_idxs

In [61]:
def update_tree(recur_prob_topic, topic_prob_topic, model, add_threshold=0.3, remove_threshold=0.1):    
    assert len(model.topic_idxs) == len(recur_prob_topic) == len(topic_prob_topic)
    update_tree_flg = False
    
    def add_topic(topic_idx, tree_idxs):
        if topic_idx in tree_idxs:
            child_idx = min([10*topic_idx+i for i in range(1, 10) if 10*topic_idx+i not in tree_idxs[topic_idx]])
            tree_idxs[topic_idx].append(child_idx)        
        else:
            child_idx = 10*topic_idx+1
            tree_idxs[topic_idx] = [10*topic_idx+1]
        return tree_idxs, child_idx
    
    added_tree_idxs = copy.deepcopy(model.tree_idxs)
    for parent_idx, child_idxs in model.tree_idxs.items():
#         rad_bow = topic_rad_bow[parent_idx]
        rad_bow = topic_prob_topic[parent_idx]
        if rad_bow > add_threshold:
            update_tree_flg = True
            for depth in range(model.tree_depth[parent_idx], model.n_depth):
                added_tree_idxs, parent_idx = add_topic(parent_idx, added_tree_idxs)
    
    def remove_topic(parent_idx, child_idx, tree_idxs):
        if parent_idx in tree_idxs:
            tree_idxs[parent_idx].remove(child_idx)
            if child_idx in tree_idxs:
                tree_idxs.pop(child_idx)    
        return tree_idxs
    
    removed_tree_idxs = copy.deepcopy(added_tree_idxs)
    for parent_idx, child_idxs in model.tree_idxs.items():
#         probs_child = np.array([topic_prob_topic[child_idx] for child_idx in child_idxs])
        probs_child = np.array([recur_prob_topic[child_idx] for child_idx in child_idxs])
#         prob_child = np.min(probs_child)
#         child_idx = child_idxs[np.argmin(probs_child)]
        for prob_child, child_idx in zip(probs_child, child_idxs):
            if prob_child < remove_threshold:
                update_tree_flg = True
                removed_tree_idxs = remove_topic(parent_idx, child_idx, removed_tree_idxs)
                if parent_idx in removed_tree_idxs:
                    if len(removed_tree_idxs[parent_idx]) == 0:
                        ancestor_idx = model.child_to_parent_idxs[parent_idx]
                        removed_tree_idxs = remove_topic(ancestor_idx, parent_idx, removed_tree_idxs)
    return removed_tree_idxs, update_tree_flg

In [62]:
losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','VALID:','TM','','',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG']]))))

In [63]:
# tree_idxs = {0:[1, 2], 
#           1:[10, 11], 2:[20, 21]}

tree_idxs = {0:[1, 2, 3], 
              1:[11, 12], 2:[21, 22], 3:[31, 32]}

# tree_idxs = {0:[1, 2, 3], 
#               1:[10, 11, 12], 2:[20, 21, 22], 3:[30, 31, 32]}


if 'sess' in globals(): sess.close()
model = Model(config, tree_idxs)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))}
saver = tf.train.Saver(max_to_keep=10)
update_tree_flg = False

In [ ]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        # validate
#         if global_step_log % config.log_period == 0:
        if global_step_log % 5000 == 0:            
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = get_loss(sess, dev_batches, model)

            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            # visualize topic
#             topic_rad_bow = {topic_idx: rad_bow for topic_idx, rad_bow in zip(model.topic_idxs, rads_bow_dev)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            
            recur_topic_idxs = {parent_idx: get_recur_topic_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in recur_topic_idxs.items()}
            
            print_topic_sample(tree_idxs, sess, model, topic_prob_topic=topic_prob_topic, topic_rad_bow=recur_prob_topic)
            print_topic_specialization(sess, model)
            print_hierarchical_affinity(sess, model)
            time_start = time.time()

            # update tree
#             tree_idxs, update_tree_flg = update_tree(topic_rad_bow, topic_prob_topic, model, add_threshold=0.2, remove_threshold=0.05)
            tree_idxs, update_tree_flg = update_tree(recur_prob_topic, topic_prob_topic, model, add_threshold=0.05, remove_threshold=0.05)
            if update_tree_flg:
                print(tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = Model(config, tree_idxs)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = get_loss(sess, dev_batches, model)
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}

recur_topic_idxs = {parent_idx: get_recur_topic_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in recur_topic_idxs.items()}
display(log_df)
print_topic_sample(tree_idxs, sess, model, topic_prob_topic=topic_prob_topic, topic_rad_bow=recur_prob_topic)

TRAIN:   TM                      VALID:   TM          \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     49   10    9  111.50  493  110.52  0.98  0.01  103.98  465  102.77   
10000    53   20   19  111.09  474  109.84  1.24  0.01  103.86  452  102.38   
15000    49   30   29  110.86  463  109.44  1.41  0.01  103.53  438  101.93   
20000    54   40   39  110.71  456  109.18  1.53  0.01  103.62  440  101.89   
25000    54   50   49  110.60  450  108.97  1.61  0.01  103.44  430  101.60   
30000    57   60   59  110.51  446  108.81  1.69  0.01  103.45  427  101.54   
35000    56   70   69  110.43  442  108.67  1.75  0.01  103.44  426  101.46   
40000    64   80   79  110.36  439  108.55  1.80  0.01  103.53  424  101.50   
45000    59   90   89  110.31  436  108.46  1.84  0.01  103.30  422  101.32   
50000    81  100   99  110.26  434  108.37  1.88  0.01  103.27  415  101.17   
55000    60  110  109  110.22  432  108.29  1.92  0.01  103.14  414  101.06   
60000    59  120  119  110.18  430  108.22  1.95  0.01  103.14  412  100.99   
65000    53  130  129  110.15  428  108.17  1.97  0.01  103.27  414  101.17   
70000    66  140  139  110.12  427  108.11  2.00  0.01  103.07  412  100.90   
75000    55  150  149  110.09  426  108.07  2.02  0.01  103.12  413  101.00   
80000    69  160  159  110.07  425  108.03  2.03  0.01  103.10  411  100.95   
85000    67  170  169  110.05  424  107.99  2.05  0.01  102.85  403  100.73   
90000    63  180  179  110.04  423  107.96  2.07  0.01  103.24  414  101.10   
95000    71  190  189  110.02  422  107.93  2.08  0.01  103.00  408  100.78   
100000   48  200  199  110.02  422  107.92  2.09  0.01  103.29  416  101.20   
105000   58  210  209  110.01  421  107.90  2.10  0.01  102.95  406  100.79   
110000   57  220  219  109.99  421  107.88  2.11  0.01  103.15  411  100.98   
115000   65  230  229  109.98  420  107.86  2.12  0.01  103.12  410  100.90   
120000   60  240  239  109.97  419  107.83  2.13  0.01  103.09  410  100.88   
125000   62  250  249  109.96  419  107.81  2.14  0.01  103.00  405  100.80   
130000   61  260  259  109.95  418  107.80  2.15  0.00  103.10  409  100.88   
135000   57  270  269  109.94  418  107.78  2.16  0.00  103.03  406  100.83   
140000   60  280  279  109.93  418  107.76  2.16  0.00  103.09  410  100.92   
145000   61  290  289  109.92  417  107.75  2.17  0.00  103.16  409  100.94   
150000   67  300  299  109.91  417  107.73  2.18  0.00  103.17  413  100.94   
155000   54  310  309  109.90  416  107.72  2.18  0.00  103.07  410  100.92   
160000   63  320  319  109.90  416  107.70  2.19  0.00  102.99  406  100.76   

                    
          KL   REG  
5000    1.20  0.00  
10000   1.47  0.00  
15000   1.60  0.00  
20000   1.72  0.00  
25000   1.84  0.01  
30000   1.91  0.00  
35000   1.98  0.00  
40000   2.03  0.00  
45000   1.98  0.00  
50000   2.10  0.00  
55000   2.08  0.00  
60000   2.14  0.00  
65000   2.10  0.00  
70000   2.17  0.00  
75000   2.13  0.00  
80000   2.15  0.00  
85000   2.12  0.00  
90000   2.14  0.00  
95000   2.21  0.00  
100000  2.09  0.00  
105000  2.16  0.00  
110000  2.17  0.00  
115000  2.22  0.00  
120000  2.20  0.00  
125000  2.20  0.00  
130000  2.22  0.00  
135000  2.20  0.00  
140000  2.17  0.00  
145000  2.22  0.00  
150000  2.23  0.00  
155000  2.15  0.00  
160000  2.23  0.00

0 R: 1.000 P: 0.421 carry pockets room pocket small back nice perfect hold work
   1 R: 0.21 P: 0.089 - inch : zipper sleeve quality nice price made bit
     13 R: 0.04 P: 0.037 ; & pro tablet surface air hard protection drive inside
     11 R: 0.09 P: 0.085 sleeve protection air protect inside pro perfectly soft ipad snug
   2 R: 0.15 P: 0.060 price bought quality ... $ amazon purchase purchased buy !
     22 R: 0.05 P: 0.054 color love ! perfectly mac pink picture perfect pro blue
     21 R: 0.03 P: 0.033 ! love perfect awesome ... recommend absolutely buy compliments highly
   3 R: 0.09 P: 0.039 months broke year weeks started month bought years zipper time
     31 R: 0.05 P: 0.054 bottom top plastic corners piece cracked part rubber feet cover
   5 R: 0.06 P: 0.021 smell bad days cheap air pay return reviews 'm item
     51 R: 0.04 P: 0.039 cover keyboard pro mac hard retina scratches air color nice
   4 R: 0.07 P: 0.027 item return received money seller amazon customer shipping or